### ASSIGNMENT 3

In [1]:
import requests
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup

Getting data From wiki

In [2]:
url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data = requests.get(url).text

# soup object
soup = BeautifulSoup(data, 'html.parser')

In [3]:
table = soup.find('table', {'class': 'wikitable sortable'})

In [4]:
columns = ['symbols' , 'security',  'GICS_sector', 'GICS_sub_industry', 'headquarters_location', 'date_added', 'founded']
sp_500 = pd.DataFrame(columns = columns)

for row in table.tbody.find_all('tr'):
    row_data = row.find_all('td')

    if row_data != []:
        symbols = row_data[0].text.strip()
        security = row_data[1].text.strip()
        gics_sector = row_data[3].text.strip()
        gics_sub_industry = row_data[4].text.strip()
        headquarters_location = row_data[5].text.strip()
        date_added = row_data[6].text.strip()
        founded = row_data[8].text.strip()

        sp_500 = sp_500.append({
            'symbols': symbols,
            'security' : security,
            'GICS_sector': gics_sector,
            'GICS_sub_industry': gics_sub_industry,
            'headquarters_location': headquarters_location,
            'date_added': date_added,
            'founded': founded}, ignore_index = True)

In [5]:
sp_500

,symbols,security,GICS_sector,GICS_sub_industry,headquarters_location,date_added,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1888
2,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,2013 (1888)
3,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,1981
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1989
...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1997
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,1969
502,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1927
503,ZION,Zions Bancorp,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,1873


Using yahoo finance lib to query api and get current stock data

In [30]:
# getting data from yahoo api
def create_stock_dataframe(ticker: str):
    tick = yf.Ticker(ticker)
    df = tick.history('id')
    return df

In [35]:
stock_price_list = []
for symbol in sp_500['symbols']:
    dt = create_stock_dataframe(symbol)
    price_list = dt.values.tolist()
    if len(price_list) == 0:
        stock_price_list.append([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, symbol])
    else:
        price_list = price_list[0]
        price_list.append(symbol)
        stock_price_list.append(price_list)

print(stock_price_list)

- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
[[174.85000610351562, 176.60000610351562, 174.25, 174.72000122070312, 2327500.0, 0.0, 0.0, 'MMM'], [135.92999267578125, 136.0, 132.0, 132.94000244140625, 8192600.0, 0.0, 0.0, 'ABT'], [125.69999694824219, 127.8499984741211, 125.37999725341797, 127.29000091552734, 6380200.0, 0.0, 0.0, 'ABBV'], [310.0, 313.9800109863281, 306.6000061035156, 311.2699890136719, 216700.0, 0.0, 0.0, 'ABMD'], [374.3800048828125, 375.7799987792969, 366.510009765625, 369.7300109863281, 2475200.0, 0.0, 0.0, 'ACN'], [58.15999984741211, 59.880001068115234, 58.150001525878906, 59.52000045776367, 8791200.0, 0.0, 0.0, 'ATVI'], [635.3599853515625, 636.0, 599.0999755859375, 614.8599853515625, 5931100.0, 0.0, 0.0, 'ADBE'], [131.6699981689453, 137.24000549316406, 130.60000610351562, 135.60000610351562, 50691000.0, 0.0, 0.0, 'AMD'], [234.1699981689453, 236.30999755859375, 233.14999389648438, 235.38999938964844, 7

In [36]:
stock_price_df = pd.DataFrame(stock_price_list, columns=['open', 'high', 'low', 'close', 'volumes', 'dividends', 'stock_splits', 'symbols'])
stock_price_df

,open,high,low,close,volumes,dividends,stock_splits,symbols
0,174.850006,176.600006,174.250000,174.720001,2327500.0,0.0,0.0,MMM
1,135.929993,136.000000,132.000000,132.940002,8192600.0,0.0,0.0,ABT
2,125.699997,127.849998,125.379997,127.290001,6380200.0,0.0,0.0,ABBV
3,310.000000,313.980011,306.600006,311.269989,216700.0,0.0,0.0,ABMD
4,374.380005,375.779999,366.510010,369.730011,2475200.0,0.0,0.0,ACN
...,...,...,...,...,...,...,...,...
500,131.559998,133.000000,131.500000,131.990005,1434500.0,0.0,0.0,YUM
501,597.340027,599.109985,582.559998,585.859985,286400.0,0.0,0.0,ZBRA
502,124.940002,126.370003,123.089996,123.279999,1291700.0,0.0,0.0,ZBH
503,61.250000,62.490002,61.119999,62.270000,1633300.0,0.0,0.0,ZION


Merging both dataframes and displaying stats of all columns

In [37]:
merged_dataframe = pd.merge(sp_500, stock_price_df, on='symbols')
merged_dataframe

,symbols,security,GICS_sector,GICS_sub_industry,headquarters_location,date_added,founded,open,high,low,close,volumes,dividends,stock_splits
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,1902,174.850006,176.600006,174.250000,174.720001,2327500.0,0.0,0.0
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1888,135.929993,136.000000,132.000000,132.940002,8192600.0,0.0,0.0
2,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,2013 (1888),125.699997,127.849998,125.379997,127.290001,6380200.0,0.0,0.0
3,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,1981,310.000000,313.980011,306.600006,311.269989,216700.0,0.0,0.0
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1989,374.380005,375.779999,366.510010,369.730011,2475200.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1997,131.559998,133.000000,131.500000,131.990005,1434500.0,0.0,0.0
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,1969,597.340027,599.109985,582.559998,585.859985,286400.0,0.0,0.0
502,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1927,124.940002,126.370003,123.089996,123.279999,1291700.0,0.0,0.0
503,ZION,Zions Bancorp,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,1873,61.250000,62.490002,61.119999,62.270000,1633300.0,0.0,0.0


In [39]:
merged_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   symbols                505 non-null    object 
 1   security               505 non-null    object 
 2   GICS_sector            505 non-null    object 
 3   GICS_sub_industry      505 non-null    object 
 4   headquarters_location  505 non-null    object 
 5   date_added             505 non-null    object 
 6   founded                505 non-null    object 
 7   open                   505 non-null    float64
 8   high                   505 non-null    float64
 9   low                    505 non-null    float64
 10  close                  505 non-null    float64
 11  volumes                505 non-null    float64
 12  dividends              505 non-null    float64
 13  stock_splits           505 non-null    float64
dtypes: float64(7), object(7)
memory usage: 59.2+ KB


In [40]:
merged_dataframe.describe(percentiles=[.25,.5,.75], include='all')

,symbols,security,GICS_sector,GICS_sub_industry,headquarters_location,date_added,founded,open,high,low,close,volumes,dividends,stock_splits
count,505,505,505,505,505,505,505,505.000000,505.000000,505.000000,505.000000,5.050000e+02,505.0,505.0
unique,505,505,11,124,258,353,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,MMM,3M,Information Technology,Health Care Equipment,"New York City, New York",,1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,75,19,44,48,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.829159,210.332412,204.900922,207.544673,4.912281e+06,0.0,0.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.350631,401.946136,391.816292,397.880818,1.108995e+07,0.0,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.919998,62.470001,60.639999,61.540001,1.023600e+06,0.0,0.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.629997,118.000000,114.290001,116.860001,1.983900e+06,0.0,0.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.949997,226.770004,224.059998,224.800003,4.517200e+06,0.0,0.0


In [42]:
merged_dataframe.to_csv("sp500_latest_stocks.csv" , index=False)